In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import icecream as ic

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torchvision.utils as vutils
from torch.utils.data import Dataset, DataLoader

from tqdm import tqdm
import wandb
from PIL import Image

e:\Programe\Anaconda\envs\Pytorch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
from icecream import ic

x = 12
x = a = x + 10

ic(a)
ic(x)


ic| a: 22
ic| x: 22


22

In [16]:
etichete_proba = torch.LongTensor(np.random.randint(0, 2, 64))
etichete_proba = torch.LongTensor([0, 0, 1, 1])

y = torch.tensor([12,10])
x = y.unsqueeze(2)
ic(etichete_proba.shape)
ic(x.shape)
ic(x)

IndexError: Dimension out of range (expected to be in range of [-2, 1], but got 2)

In [19]:
embeding_size = 50
attribute_number = 2
embedd = nn.Embedding(attribute_number, embeding_size)
labels = torch.tensor([0, 0, 1, 1,1,0])

embedded_labels = embedd(labels).unsqueeze(2).unsqueeze(3)
print(embedded_labels.shape)


torch.Size([6, 50, 1, 1])


In [21]:
x = torch.randn(4, 3,64,64)  
ic(x.shape)


ic| x.shape: torch.Size([4, 3, 64, 64])


torch.Size([4, 3, 64, 64])

In [41]:
from dataset import DatasetCelebA, DatasetCelebA_Sketch

EXCEL_PATH = "E:\Lucru\Dizertatie\Cod\Face_Synthesis\Database\celebA_small.xlsx"
DATASET_PATH = "E:\Lucru\Dizertatie\Cod\Face_Synthesis\Database\small_dataset\\"
SKETCH_DATASET_PATH = "E:\Lucru\Dizertatie\Cod\Face_Synthesis\Database\small_dataset_sketch\\"

dataset = DatasetCelebA_Sketch(base_path=DATASET_PATH, excel_path=EXCEL_PATH, sketch_path=SKETCH_DATASET_PATH)

index = 0
image, sketch, label = dataset[0]
image2, sketch2, label2 = dataset[2]

esantioane_proba = torch.stack([image, image2], dim=0)

ic(esantioane_proba.shape)
ic(etichete_proba.shape)


ic| image.shape: torch.Size([3, 64, 64])
ic| sketch.shape: torch.Size([3, 64, 64])
ic| label: 0


RuntimeError: stack expects each tensor to be equal size, but got [0] at entry 0 and [1] at entry 1